In [1]:
Aromodel_Path = "/home/robert/research/aromodel"
lib_path = Aromodel_Path + "/code"

import sys
sys.path.append(lib_path)
sys.path

['/home/robert/research/impropers_analysis/impropers_angles_data',
 '/home/robert/mambaforge/envs/simulations/lib/python311.zip',
 '/home/robert/mambaforge/envs/simulations/lib/python3.11',
 '/home/robert/mambaforge/envs/simulations/lib/python3.11/lib-dynload',
 '',
 '/home/robert/mambaforge/envs/simulations/lib/python3.11/site-packages',
 '/home/robert/research/aromodel/code']

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import Atom
import Molecule
import Ring
import OPLS as op
import System
import Conjugated_Polymer
import Cluster_IO
import Write_Inputs
import Write_Submit_Script
import copy
import scipy
import os
import sys
import matplotlib.cm
import matplotlib as mpl
import re
import scipy
from aromodel_lib import Read_Input

"""This is the script used to make Figure 3 of the
Impropers draft (comparison of frequency of improper 
angles in films when comparing LigParGen angle
parameters and Geometry-Optimized angle 
parameters. The reason that this matters is
because standard angle parameters allow lower
energetic penalty for improper torsion, as they
do not add up to 360 degrees and thus are more flexible
and have minima with some actual improper torsion)"""

sys.setrecursionlimit(10000)

In [4]:
Polymer_List = ["P3HT","PTB7","PNDI-T"]
Dump_List = ["P3HT","PTB7","PNDI-T"]
Dih_File_List = ["P3HT","PTB7","PNDI-T"]
# Input_Files = ["P3HT_Input_With_Sidechains.txt","PTB7_Input_With_Sidechains.txt","PNDI_T_Input_With_Sidechains.txt"]
# XYZ_Files = ["P3HT_Input_With_Sidechains.xyz","PTB7_Input_With_Sidechains.xyz","PNDI_T_Input_With_Sidechains.xyz"]
Input_Files = ["P3HT_Input.txt","PTB7_Input.txt","PNDI_T_Input.txt"]
XYZ_Files = ["P3HT_Input.xyz","PTB7_Input.xyz","PNDI_T_Input.xyz"]

In [5]:
def Read_Input(Input_File,XYZ_File,Polymer_Name):
#This function reads in the parameter file, assigns the atoms to rings, adds available LJ, coulombic, bonded, angular, dihedral, and improper potentials, and tells the program whether it needs to parameterize missing bond potentials or partial charges. Returns Ring_List: NumPy array of Ring objects categorizing all available atoms into separate rings; Paramaterize_Bond: Boolean that equals "True" if bond parameters for interring bonds have not been specified; Paramaterize_Charges: Boolean that equals "True" if partial charges for atoms have not been specified
    Supermolecule = Molecule.Molecule(XYZ_File)
    #Open file and read lines
    f = open(Input_File)
    lines = f.readlines()
    Ring_List = []
    Ring_Read = False
    Bonded_Read = False
    Aux_Read = False
    Plumed_List = [[],[]]
    for line in lines:
        if len(line.strip().split()) > 0 and line.strip().split()[0] == "***":
            Bonded_Read = False
            for b_atoms in Bonded_Atom_List:
                Bonded_Atom_Vectors.append(Supermolecule.Get_Atom(b_atoms[1]).Position - Supermolecule.Get_Atom(b_atoms[0]).Position)
            New_Ring = Ring.Ring(Supermolecule,Ring_Atom_List,Core_Atom_List,Bonded_Atom_List,Bonded_Atom_Vectors,Ring_Name,Ring_ID,Polymer_Name,Ring_Nickname,Symmetric = Symmetry,Aux_Ring_List=Aux_Rings,Plumed_Rings=Plumed_List)
            Atom_Plumed_List = [[],[]]
            for x,ap_list in enumerate(Plumed_List):
                for atom in ap_list:
                    Atom_Plumed_List[x].append(Supermolecule.Get_Atom(atom))
            #New_Ring.Add_Plumed_Rings(Atom_Plumed_List)
            Plumed_List = [[],[]]
            Ring_List.append(New_Ring)
        if Bonded_Read:
            String_Bonded_Atom_List = line.strip().split()
            Bonded_Atom_List.append([int(i) for i in String_Bonded_Atom_List])
        if Aux_Read:
            if Aux_Count <= Aux_Ring_Num:
                Aux_Rings.append([[],[],int(line.strip().split()[1]),int(line.strip().split()[2]),line.strip().split()[0],line.strip().split()[3]])
                Aux_Count += 1
            else:
                Aux_Read = False
                Ring_Read = True
        if Ring_Read:
            if len(line.strip().split()) == 1 or "CORE" in line.strip().split()[-1]:
                Ring_Atom_List.append(int(line.strip().split()[0]))
                if "CORE" in line.strip().split()[-1]:
                    Core_Atom_List.append(int(line.strip().split()[0]))
                    if line.strip().split()[-1] == "CORE1" or line.strip().split()[-2] == "CORE1":
                        Plumed_List[0].append(int(line.strip().split()[0]))
                    if line.strip().split()[-1] == "CORE2":
                        Plumed_List[1].append(int(line.strip().split()[0]))
            elif line.strip().split()[-1][:3] == "AUX":
                Aux_Rings[int(line.strip().split()[-1][-1])][0].append(int(line.strip().split()[0]))
                if len(line.strip().split()[-1]) > 5:
                    Aux_Rings[int(line.strip().split()[-1][-1])][1].append(int(line.strip().split()[0]))
            elif line.strip().split()[0] == "Bonded":
                Ring_Read = False
                Bonded_Read = True
        if len(line.strip().split()) > 0 and line.strip().split()[0] == "Ring":
            Ring_ID = int(line.strip().split()[1])
            if line.strip().split()[2].strip() == "True":
                Symmetry = True
            else:
                Symmetry = False
            Ring_Name = line.strip().split()[3].strip()
            Ring_Nickname = line.strip().split()[4].strip()
            Aux_Ring_Num = int(line.strip().split()[5].strip())
            if Aux_Ring_Num != 0:
                Aux_Read = True
                Aux_Count = 1
            else:
                Ring_Read = True
            Ring_Atom_List = []
            Bonded_Atom_List = []
            Bonded_Atom_Vectors = []
            Core_Atom_List = []
            Aux_Rings = []
            #if Aux_Ring_Num != 0:

    return Ring_List,False,False

In [27]:
for poly,dump,dih_f,Input_File,XYZ_File in zip(Polymer_List,Dump_List,Dih_File_List,Input_Files,XYZ_Files):

    Sub_Directory_1 = "./"
    Sub_Directory_2 = "./"
    Modified_DIH_List = []
    Original_DIH_List = []
    Polymer_Name = Input_File.split('_With')[0]

    #Get impropers of interest from mock read in
    Ring_List,Parameterize_Bond,Parameterize_Charges = Read_Input(Input_File,XYZ_File,Polymer_Name)
    #Manually run LigParGen, then you're ready for:
    for i,pring in enumerate(Ring_List):
        pring.Read_In_LigParGen(i)
    #Now create a list of all the rings you want to include
    Polymer_Ring_List = []
    for i in range(15):
        for ring in Ring_List:
            Polymer_Ring_List.append(copy.deepcopy(ring))
    #Connect the rings together
    for i in range(14):
        if (not Polymer_Ring_List[i].Bonded_Atoms[-1].Is_Linked) and not (Polymer_Ring_List[i+1].Bonded_Atoms[0].Is_Linked):
            Polymer_Ring_List[i].Bonded_Atoms[-1].Add_Ring(Polymer_Ring_List[i+1])
            Polymer_Ring_List[i+1].Bonded_Atoms[0].Add_Ring(Polymer_Ring_List[i])
            Polymer_Ring_List[i].Bonded_Atoms[-1].Add_Bonded_Atom(Polymer_Ring_List[i+1].Bonded_Atoms[0])
            Polymer_Ring_List[i+1].Bonded_Atoms[0].Add_Bonded_Atom(Polymer_Ring_List[i+1].Bonded_Atoms[-1])
    #Create a system of 30 of those polymers and print a LAMMPS data file
    Polymer = Conjugated_Polymer.Conjugated_Polymer(Polymer_Ring_List)
    System_List = []
    Comp_List = np.ones(30,dtype=int)
    for polym in range(30):
        System_List.append(copy.deepcopy(Polymer))
    Film = System.System(System_List,Comp_List,300.0,"%s_Impropers" % poly)
    Film.Gen_Rand()
    Film.Write_LAMMPS_Data()

    Interring_Impropers = []
    for mol in Film.Molecule_List:
        for ring in mol.Ring_List:
            for b_atom in ring.Bonded_Atoms:
                if b_atom.Is_Linked:
                    Interring_Impropers.append([b_atom.Central_Atom.System_ID,b_atom.Same_Ring_Bonded_Atom_List[0].System_ID,b_atom.Same_Ring_Bonded_Atom_List[1].System_ID,b_atom.Interring_Bond_Atom.Central_Atom.System_ID])

    print(Interring_Impropers)
    #Get improper measurements from worked up file and add them if they are interring
    torsion_index = 1
    torsion_indices = []
    for polymer_num in range(1):
        torsion_indices.append(torsion_index)
        torsion_index += 1

    for snapshot_num in torsion_indices:
        print(snapshot_num)
        f = open(Sub_Directory_1 + "/Improper_Lists_CombinedAngles_%s.dump" % dump,'r')
        Temp_Dihs = f.readlines()
        f.close()
        Temp_Dihs = Temp_Dihs[9:]
        Dihs = []
        for line in Temp_Dihs:
            Dihs.append(line.strip().split())
        for dih in Interring_Impropers:
            for file_dih in Dihs:
                if int(file_dih[2]) == dih[0] and int(file_dih[3]) == dih[1] and int(file_dih[4]) == dih[2] and int(file_dih[5]) == dih[3] or int(file_dih[2]) == dih[0] and int(file_dih[4]) == dih[1] and int(file_dih[3]) == dih[2] and int(file_dih[5]) == dih[3]:
                    for opposite_dih in Interring_Impropers:
                        if opposite_dih[3] == dih[0]:
                            for file_dih_2 in Dihs:
                                if int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[3]) == opposite_dih[1] and int(file_dih_2[4]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3] or int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[4]) == opposite_dih[1] and int(file_dih_2[3]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3]:
                                    #print(file_dih)
                                    Modified_DIH_List.append(abs(float(file_dih[-1])) + abs(float(file_dih_2[-1])))
                                    break
        
    #Get colvar measurements from conv file

    for snapshot_num in torsion_indices:
        print(snapshot_num)
        f = open(Sub_Directory_2 + "/Improper_Lists_LigParGen_%s.dump" % dump,'r')
        Temp_Dihs = f.readlines()
        f.close()
        Temp_Dihs = Temp_Dihs[9:]
        Dihs = []
        for line in Temp_Dihs:
            Dihs.append(line.strip().split())
        for dih in Interring_Impropers:
            for file_dih in Dihs:
                if int(file_dih[2]) == dih[0] and int(file_dih[3]) == dih[1] and int(file_dih[4]) == dih[2] and int(file_dih[5]) == dih[3] or int(file_dih[2]) == dih[0] and int(file_dih[3]) == dih[1] and int(file_dih[5]) == dih[2] and int(file_dih[3]) == dih[3]:
                    for opposite_dih in Interring_Impropers:
                        if opposite_dih[3] == dih[0]:
                            for file_dih_2 in Dihs:
                                if int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[3]) == opposite_dih[1] and int(file_dih_2[4]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3] or int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[4]) == opposite_dih[1] and int(file_dih_2[3]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3]:
                                    #print(file_dih)
                                    Original_DIH_List.append(abs(float(file_dih[-1])) + abs(float(file_dih_2[-1])))
                                    break

    num_bins = 360
    bins_x_dih = np.linspace(-180,180,num_bins)
    bins_y_dih = np.zeros(num_bins)
    bins_y_raw_mod = np.zeros(num_bins)
    bins_y_raw_orig = np.zeros(num_bins)

    for dih in Modified_DIH_List:
        for count in range(len(bins_y_dih)):
            if dih >= -180 + (count * (2*180/num_bins)) and dih < -180 + ((count+1) * (2*180/num_bins)):
                bins_y_dih[count] += 1
                bins_y_raw_mod[count] += 1
                break

    for dih in Original_DIH_List:
        for count in range(len(bins_y_dih)):
            if dih >= -180 + (count * (2*180/num_bins)) and dih < -180 + ((count+1) * (2*180/num_bins)):
                bins_y_dih[count] -= 1
                bins_y_raw_orig[count] -= 1
                break

    bins_y_dih = bins_y_dih / 2
    bins_y_raw_orig = bins_y_raw_orig / 2
    bins_y_raw_mod = bins_y_raw_mod / 2
    print(Modified_DIH_List)
    print(Original_DIH_List)
    fig,ax = plt.subplots(1,1)
    plt.bar(bins_x_dih,bins_y_dih,width=1,color='b',edgecolor='b',label='Dihedrals')
    plt.legend(fontsize="x-large")
    ax.tick_params(axis="x", labelsize=24)
    ax.tick_params(axis="y", labelsize=24)
    ax.tick_params(length=4,width=4)
    plt.tick_params(axis='x',which='both',bottom=False,top=False)
    plt.xticks(np.arange(-180, 180+.001, step=90),[-180,-90,'0',90,180])
    plt.yticks(np.arange(-60, 60, step=20))
    plt.xlim(-180,180)
    plt.ylim(-100,100)
    plt.xlabel('Angle (degrees)',size=24)
    plt.ylabel('Frequency',size=24)
    plt.tight_layout()
    fig.savefig('%s_Improper_Comparsion' % dump)
    plt.close(fig)


    #collapse from 360 to 180
    Condensed_List = []
    for bin1,bin2 in zip(bins_y_dih[:int(math.ceil(len(bins_x_dih)/2))],bins_y_dih[:int(math.floor(len(bins_x_dih)/2)):-1]):
        Condensed_List.append(bin1 + bin2)
    Condensed_List.append(bins_y_dih[int(len(bins_x_dih)/2)])

    Condensed_Raw_Orig = []
    for bin1,bin2 in zip(bins_y_raw_orig[:int(math.ceil(len(bins_x_dih)/2))],bins_y_raw_orig[:int(math.floor(len(bins_x_dih)/2)):-1]):
        Condensed_Raw_Orig.append(bin1 + bin2)
    Condensed_Raw_Orig.append(bins_y_raw_orig[int(len(bins_x_dih)/2)])

    Condensed_Raw_Mod = []
    for bin1,bin2 in zip(bins_y_raw_mod[:int(math.ceil(len(bins_x_dih)/2))],bins_y_raw_mod[:int(math.floor(len(bins_x_dih)/2)):-1]):
        Condensed_Raw_Mod.append(bin1 + bin2)
    Condensed_Raw_Mod.append(bins_y_raw_mod[int(len(bins_x_dih)/2)])


    #collapse from 180 to 90
    Condensed_List_2 = []
    for bin1,bin2 in zip(Condensed_List[:int(math.ceil(len(Condensed_List)/2))],Condensed_List[:int(math.floor(len(Condensed_List)/2)):-1]):
        Condensed_List_2.append(bin1 + bin2)
    Condensed_List_2.append(bins_y_dih[int(len(bins_x_dih)/4)])

    Condensed_Raw_Mod_2 = []
    for bin1,bin2 in zip(Condensed_Raw_Mod[:int(math.ceil(len(Condensed_Raw_Mod)/2))],Condensed_Raw_Mod[:int(math.floor(len(Condensed_Raw_Mod)/2)):-1]):
        Condensed_Raw_Mod_2.append(bin1 + bin2)
    Condensed_Raw_Mod_2.append(bins_y_dih[int(len(bins_x_dih)/4)])

    Condensed_Raw_Orig_2 = []
    for bin1,bin2 in zip(Condensed_Raw_Orig[:int(math.ceil(len(Condensed_Raw_Orig)/2))],Condensed_Raw_Orig[:int(math.floor(len(Condensed_Raw_Orig)/2)):-1]):
        Condensed_Raw_Orig_2.append(bin1 + bin2)
    Condensed_Raw_Orig_2.append(bins_y_dih[int(len(bins_x_dih)/4)])

    bins_x_dih = np.linspace(0,90,int(num_bins/4))

    for newbin in Condensed_List_2:
        if newbin > 0:
            newbin = 0

    fig,ax = plt.subplots(1,1)
    plt.bar(bins_x_dih,Condensed_List_2,width=1,color='r',edgecolor='r',label='LigParGen Angles')
    for i in range(len(Condensed_List_2)):
        if Condensed_List_2[i] < 0:
            Condensed_List_2[i] = 0
    plt.bar(bins_x_dih,Condensed_List_2,width=1,color='b',edgecolor='b',label='Geometry-Optimized Angles')
    plt.legend(fontsize="x-large")
    ax.tick_params(axis="x", labelsize=24)
    ax.tick_params(axis="y", labelsize=24)
    ax.tick_params(length=4,width=4)
    plt.tick_params(axis='x',which='both',bottom=False,top=False)
    plt.xticks(np.arange(0, 20+.001, step=5))
    plt.yticks(np.arange(-100, 101, step=50),[100,50,0,50,100])
    plt.xlim(0,20)
    plt.ylim(-50,51)
    plt.xlabel('Improper Angle (degrees)',size=24)
    plt.ylabel('Relative Frequency',size=24)
    plt.tight_layout()
    fig.savefig('%s_Improper_Comparsion_Condensed' % dump)
    plt.close(fig)

    #raw figures
    fig,ax = plt.subplots(1,1)
    print(Condensed_Raw_Orig_2)
    print(Condensed_Raw_Mod_2)
    plt.bar(bins_x_dih,Condensed_Raw_Orig_2,width=1,color='r',edgecolor='r',label='LigParGen Angles')
    plt.bar(bins_x_dih,Condensed_Raw_Mod_2,width=1,color='b',edgecolor='b',label='Geometry-Optimized Angles')
    plt.legend(fontsize="x-large")
    ax.tick_params(axis="x", labelsize=24)
    ax.tick_params(axis="y", labelsize=24)
    ax.tick_params(length=4,width=4)
    plt.tick_params(axis='x',which='both',bottom=False,top=False)
    plt.xticks(np.arange(0, 20+.001, step=10))
    plt.yticks(np.arange(-250, 251, step=125),[250,125,0,125,250])
    plt.xlim(0,20)
    plt.ylim(-150,151)
    plt.xlabel('Improper Angle (degrees)',size=24)
    plt.ylabel('Absolute Frequency',size=24)
    plt.tight_layout()
    fig.savefig('%s_Improper_Comparsion_Condensed_Raw' % dump)
    plt.close(fig)

Writing slurm; File_Name: sub_P3HT_Input.txt_Methylthiophene_1_Optimize_Monomer, In_File: P3HT_Input.txt_Methylthiophene_1_Optimize_Monomer.inp, Job_Name: P3HT_Input.txt_Methylthiophene_1_Optimize_Monomer
Writing slurm; File_Name: sub_P3HT_Input.txt_Methylthiophene_2_Optimize_Monomer, In_File: P3HT_Input.txt_Methylthiophene_2_Optimize_Monomer.inp, Job_Name: P3HT_Input.txt_Methylthiophene_2_Optimize_Monomer
Number of Molecules =  30
Genenerated 1 Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthio

/home/rramji/.bashrc: line 3: module: command not found
/home/rramji/.bashrc: line 4: module: command not found


copying 2 files to the cluster, starting with sub_PTB7_Input.txt_Dimethoxy_Benzodithiophene_1_Optimize_Monomer


/home/rramji/.bashrc: line 3: module: command not found
/home/rramji/.bashrc: line 4: module: command not found


in Cluster_IO.submit_Job, I'm attempting to use sbatch sub_PTB7_Input.txt_Dimethoxy_Benzodithiophene_1_Optimize_Monomer


/home/rramji/.bashrc: line 3: module: command not found
/home/rramji/.bashrc: line 4: module: command not found


Submitted batch job 28289481
started return_info, in ring.py
looking for ./Optimized_Monomers/PTB7_Input.txt_Dimethoxy_Benzodithiophene_1_Optimize_Monomer.out in Clulster_IO.Return_Info


dos2unix: converting file sub_PTB7_Input.txt_Dimethoxy_Benzodithiophene_1_Optimize_Monomer to Unix format...
/home/rramji/.bashrc: line 3: module: command not found
/home/rramji/.bashrc: line 4: module: command not found
/home/rramji/.bashrc: line 3: module: command not found
/home/rramji/.bashrc: line 4: module: command not found


taking a nap in Cluster_IO.Return_Info while the job runs, check back in a couple minutes


In [25]:
np.arange(-100, 101, step=50)

array([-100,  -50,    0,   50,  100])

In [20]:
Condensed_List.append(bins_y_dih[int(len(bins_x_dih)/2)])

In [3]:
Polymer_List = ["P3HT"]
Dump_List = ["P3HT"]
Dih_File_List = ["P3HT"]
Input_Files = ["P3HT_Input.txt"]
XYZ_Files = ["P3HT_Input.xyz"]

In [5]:
poly = "P3HT_Input_With_Sidechains"
dump = "P3HT_Input_With_Sidechains" 
dih_f = "P3HT_Input_With_Sidechains" 
Input_File = "P3HT_Input_Input_With_Sidechains.txt"
XYZ_File = "P3HT_Input_Input_With_Sidechains.xyz"

In [ ]:
Sub_Directory_1 = "./"
Sub_Directory_2 = "./"
Modified_DIH_List = []
Original_DIH_List = []
Polymer_Name = Input_File.split('_With')[0]

#Get impropers of interest from mock read in
Ring_List,Parameterize_Bond,Parameterize_Charges = Read_Input(Input_File,XYZ_File,Polymer_Name)
#Manually run LigParGen, then you're ready for:
for i,pring in enumerate(Ring_List):
    pring.Read_In_LigParGen(i)
#Now create a list of all the rings you want to include
Polymer_Ring_List = []
for i in range(15):
    for ring in Ring_List:
        Polymer_Ring_List.append(copy.deepcopy(ring))

In [4]:
for poly,dump,dih_f,Input_File,XYZ_File in zip(Polymer_List,Dump_List,Dih_File_List,Input_Files,XYZ_Files):


    #Connect the rings together
    for i in range(14):
        if (not Polymer_Ring_List[i].Bonded_Atoms[-1].Is_Linked) and not (Polymer_Ring_List[i+1].Bonded_Atoms[0].Is_Linked):
            Polymer_Ring_List[i].Bonded_Atoms[-1].Add_Ring(Polymer_Ring_List[i+1])
            Polymer_Ring_List[i+1].Bonded_Atoms[0].Add_Ring(Polymer_Ring_List[i])
            Polymer_Ring_List[i].Bonded_Atoms[-1].Add_Bonded_Atom(Polymer_Ring_List[i+1].Bonded_Atoms[0])
            Polymer_Ring_List[i+1].Bonded_Atoms[0].Add_Bonded_Atom(Polymer_Ring_List[i+1].Bonded_Atoms[-1])
    #Create a system of 30 of those polymers and print a LAMMPS data file
    Polymer = Conjugated_Polymer.Conjugated_Polymer(Polymer_Ring_List)
    System_List = []
    Comp_List = np.ones(30,dtype=int)
    for polym in range(30):
        System_List.append(copy.deepcopy(Polymer))
    Film = System.System(System_List,Comp_List,300.0,"%s_Impropers" % poly)
    Film.Gen_Rand()
    Film.Write_LAMMPS_Data()ls 

    Interring_Impropers = []
    for mol in Film.Molecule_List:
        for ring in mol.Ring_List:
            for b_atom in ring.Bonded_Atoms:
                if b_atom.Is_Linked:
                    Interring_Impropers.append([b_atom.Central_Atom.System_ID,b_atom.Same_Ring_Bonded_Atom_List[0].System_ID,b_atom.Same_Ring_Bonded_Atom_List[1].System_ID,b_atom.Interring_Bond_Atom.Central_Atom.System_ID])

    print(Interring_Impropers)
    #Get improper measurements from worked up file and add them if they are interring
    torsion_index = 1
    torsion_indices = []
    for polymer_num in range(1):
        torsion_indices.append(torsion_index)
        torsion_index += 1

    for snapshot_num in torsion_indices:
        print(snapshot_num)
        f = open(Sub_Directory_1 + "/Improper_Lists_CombinedAngles_%s.dump" % dump,'r')
        Temp_Dihs = f.readlines()
        f.close()
        Temp_Dihs = Temp_Dihs[9:]
        Dihs = []
        for line in Temp_Dihs:
            Dihs.append(line.strip().split())
        for dih in Interring_Impropers:
            for file_dih in Dihs:
                if int(file_dih[2]) == dih[0] and int(file_dih[3]) == dih[1] and int(file_dih[4]) == dih[2] and int(file_dih[5]) == dih[3] or int(file_dih[2]) == dih[0] and int(file_dih[4]) == dih[1] and int(file_dih[3]) == dih[2] and int(file_dih[5]) == dih[3]:
                    for opposite_dih in Interring_Impropers:
                        if opposite_dih[3] == dih[0]:
                            for file_dih_2 in Dihs:
                                if int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[3]) == opposite_dih[1] and int(file_dih_2[4]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3] or int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[4]) == opposite_dih[1] and int(file_dih_2[3]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3]:
                                    #print(file_dih)
                                    Modified_DIH_List.append(abs(float(file_dih[-1])) + abs(float(file_dih_2[-1])))
                                    break
        
    #Get colvar measurements from conv file

    for snapshot_num in torsion_indices:
        print(snapshot_num)
        f = open(Sub_Directory_2 + "/Improper_Lists_LigParGen_%s.dump" % dump,'r')
        Temp_Dihs = f.readlines()
        f.close()
        Temp_Dihs = Temp_Dihs[9:]
        Dihs = []
        for line in Temp_Dihs:
            Dihs.append(line.strip().split())
        for dih in Interring_Impropers:
            for file_dih in Dihs:
                if int(file_dih[2]) == dih[0] and int(file_dih[3]) == dih[1] and int(file_dih[4]) == dih[2] and int(file_dih[5]) == dih[3] or int(file_dih[2]) == dih[0] and int(file_dih[3]) == dih[1] and int(file_dih[5]) == dih[2] and int(file_dih[3]) == dih[3]:
                    for opposite_dih in Interring_Impropers:
                        if opposite_dih[3] == dih[0]:
                            for file_dih_2 in Dihs:
                                if int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[3]) == opposite_dih[1] and int(file_dih_2[4]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3] or int(file_dih_2[2]) == opposite_dih[0] and int(file_dih_2[4]) == opposite_dih[1] and int(file_dih_2[3]) == opposite_dih[2] and int(file_dih_2[5]) == opposite_dih[3]:
                                    #print(file_dih)
                                    Original_DIH_List.append(abs(float(file_dih[-1])) + abs(float(file_dih_2[-1])))
                                    break

    num_bins = 360
    bins_x_dih = np.linspace(-180,180,num_bins)
    bins_y_dih = np.zeros(num_bins)
    bins_y_raw_mod = np.zeros(num_bins)
    bins_y_raw_orig = np.zeros(num_bins)

    for dih in Modified_DIH_List:
        for count in range(len(bins_y_dih)):
            if dih >= -180 + (count * (2*180/num_bins)) and dih < -180 + ((count+1) * (2*180/num_bins)):
                bins_y_dih[count] += 1
                bins_y_raw_mod[count] += 1
                break

    for dih in Original_DIH_List:
        for count in range(len(bins_y_dih)):
            if dih >= -180 + (count * (2*180/num_bins)) and dih < -180 + ((count+1) * (2*180/num_bins)):
                bins_y_dih[count] -= 1
                bins_y_raw_orig[count] -= 1
                break

    bins_y_dih = bins_y_dih / 2
    bins_y_raw_orig = bins_y_raw_orig / 2
    bins_y_raw_mod = bins_y_raw_mod / 2
    print(Modified_DIH_List)
    print(Original_DIH_List)
    fig,ax = plt.subplots(1,1)
    plt.bar(bins_x_dih,bins_y_dih,width=1,color='b',edgecolor='b',label='Dihedrals')
    plt.legend(fontsize="x-large")
    ax.tick_params(axis="x", labelsize=24)
    ax.tick_params(axis="y", labelsize=24)
    ax.tick_params(length=4,width=4)
    plt.tick_params(axis='x',which='both',bottom=False,top=False)
    plt.xticks(np.arange(-180, 180+.001, step=90),[-180,-90,'0',90,180])
    plt.yticks(np.arange(-60, 60, step=20))
    plt.xlim(-180,180)
    plt.ylim(-100,100)
    plt.xlabel('Angle (degrees)',size=24)
    plt.ylabel('Frequency',size=24)
    plt.tight_layout()
    fig.savefig('%s_Improper_Comparsion' % dump)
    plt.close(fig)


    #collapse from 360 to 180
    Condensed_List = []
    for bin1,bin2 in zip(bins_y_dih[:int(math.ceil(len(bins_x_dih)/2))],bins_y_dih[:int(math.floor(len(bins_x_dih)/2)):-1]):
        Condensed_List.append(bin1 + bin2)
    Condensed_List.append(bins_y_dih[int(len(bins_x_dih)/2)])

    Condensed_Raw_Orig = []
    for bin1,bin2 in zip(bins_y_raw_orig[:int(math.ceil(len(bins_x_dih)/2))],bins_y_raw_orig[:int(math.floor(len(bins_x_dih)/2)):-1]):
        Condensed_Raw_Orig.append(bin1 + bin2)
    Condensed_Raw_Orig.append(bins_y_raw_orig[int(len(bins_x_dih)/2)])

    Condensed_Raw_Mod = []
    for bin1,bin2 in zip(bins_y_raw_mod[:int(math.ceil(len(bins_x_dih)/2))],bins_y_raw_mod[:int(math.floor(len(bins_x_dih)/2)):-1]):
        Condensed_Raw_Mod.append(bin1 + bin2)
    Condensed_Raw_Mod.append(bins_y_raw_mod[int(len(bins_x_dih)/2)])


    #collapse from 180 to 90
    Condensed_List_2 = []
    for bin1,bin2 in zip(Condensed_List[:int(math.ceil(len(Condensed_List)/2))],Condensed_List[:int(math.floor(len(Condensed_List)/2)):-1]):
        Condensed_List_2.append(bin1 + bin2)
    Condensed_List_2.append(bins_y_dih[int(len(bins_x_dih)/4)])

    Condensed_Raw_Mod_2 = []
    for bin1,bin2 in zip(Condensed_Raw_Mod[:int(math.ceil(len(Condensed_Raw_Mod)/2))],Condensed_Raw_Mod[:int(math.floor(len(Condensed_Raw_Mod)/2)):-1]):
        Condensed_Raw_Mod_2.append(bin1 + bin2)
    Condensed_Raw_Mod_2.append(bins_y_dih[int(len(bins_x_dih)/4)])

    Condensed_Raw_Orig_2 = []
    for bin1,bin2 in zip(Condensed_Raw_Orig[:int(math.ceil(len(Condensed_Raw_Orig)/2))],Condensed_Raw_Orig[:int(math.floor(len(Condensed_Raw_Orig)/2)):-1]):
        Condensed_Raw_Orig_2.append(bin1 + bin2)
    Condensed_Raw_Orig_2.append(bins_y_dih[int(len(bins_x_dih)/4)])

    bins_x_dih = np.linspace(0,90,int(num_bins/4))

    for newbin in Condensed_List_2:
        if newbin > 0:
            newbin = 0

    fig,ax = plt.subplots(1,1)
    plt.bar(bins_x_dih,Condensed_List_2,width=1,color='r',edgecolor='r',label='LigParGen Angles')
    for i in range(len(Condensed_List_2)):
        if Condensed_List_2[i] < 0:
            Condensed_List_2[i] = 0
    plt.bar(bins_x_dih,Condensed_List_2,width=1,color='b',edgecolor='b',label='Geometry-Optimized Angles')
    plt.legend(fontsize="x-large")
    ax.tick_params(axis="x", labelsize=24)
    ax.tick_params(axis="y", labelsize=24)
    ax.tick_params(length=4,width=4)
    plt.tick_params(axis='x',which='both',bottom=False,top=False)
    plt.xticks(np.arange(0, 20+.001, step=5))
    plt.yticks(np.arange(-100, 101, step=50),[100,50,0,50,100])
    plt.xlim(0,20)
    plt.ylim(-50,51)
    plt.xlabel('Improper Angle (degrees)',size=24)
    plt.ylabel('Relative Frequency',size=24)
    plt.tight_layout()
    fig.savefig('%s_Improper_Comparsion_Condensed' % dump)
    plt.close(fig)

    #raw figures
    fig,ax = plt.subplots(1,1)
    print(Condensed_Raw_Orig_2)
    print(Condensed_Raw_Mod_2)
    plt.bar(bins_x_dih,Condensed_Raw_Orig_2,width=1,color='r',edgecolor='r',label='LigParGen Angles')
    plt.bar(bins_x_dih,Condensed_Raw_Mod_2,width=1,color='b',edgecolor='b',label='Geometry-Optimized Angles')
    plt.legend(fontsize="x-large")
    ax.tick_params(axis="x", labelsize=24)
    ax.tick_params(axis="y", labelsize=24)
    ax.tick_params(length=4,width=4)
    plt.tick_params(axis='x',which='both',bottom=False,top=False)
    plt.xticks(np.arange(0, 20+.001, step=10))
    plt.yticks(np.arange(-250, 251, step=125),[250,125,0,125,250])
    plt.xlim(0,20)
    plt.ylim(-150,151)
    plt.xlabel('Improper Angle (degrees)',size=24)
    plt.ylabel('Absolute Frequency',size=24)
    plt.tight_layout()
    fig.savefig('%s_Improper_Comparsion_Condensed_Raw' % dump)
    plt.close(fig)

Number of atoms in structure file: 42
Writing slurm; File_Name: sub_P3HT_Input.txt_Methylthiophene_1_Optimize_Monomer, In_File: P3HT_Input.txt_Methylthiophene_1_Optimize_Monomer.inp, Job_Name: P3HT_Input.txt_Methylthiophene_1_Optimize_Monomer
10
Writing slurm; File_Name: sub_P3HT_Input.txt_Methylthiophene_2_Optimize_Monomer, In_File: P3HT_Input.txt_Methylthiophene_2_Optimize_Monomer.inp, Job_Name: P3HT_Input.txt_Methylthiophene_2_Optimize_Monomer
10
Number of Molecules =  30
Genenerated 1 Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Theta_0_Methylthiophene_Phi_0_Thet